# MNE-Python: From raw data to epochs and evoked responses (ERF/ERP)

`Authors:
 Annalisa Pascarella
 Vanessa Hadid`

In [ ]:
import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

## Load the mne package


In [ ]:
import mne
print(mne.__version__)

We set the logging level to 'warning' so the output will be less verbose

In [ ]:
mne.set_log_level('warning')

## Download sample dataset

Now we import the [`sample`](https://mne.tools/stable/documentation/datasets.html#sample-dataset) dataset. It will be downloaded automatically (approx. 2 GB)

In [ ]:
from mne.datasets import sample
data_path = sample.data_path()

raw_fname = os.path.join(data_path, 'MEG/sample/sample_audvis_filt-0-40_raw.fif')
print(raw_fname)

## Read data from file

To see what a function does...use this notation!


In [ ]:
mne.io.read_raw_fif?

In [ ]:
raw = mne.io.read_raw_fif(raw_fname, preload=True)
print(raw)

Printing the [Raw](https://mne.tools/stable/generated/mne.io.Raw.html#mne.io.Raw) object displays some basic information like the total number of channels, the number of time points at which the data were sampled, total duration, and the approximate size in memory. 

Note that by default, the data will actually not be loaded into memory automatically to preserve memory. To actually load the data, we have to pass `preload=True`.

Now let's look at the measurement info. There is also quite a lot of information stored in the `raw.info` attribute.  It will give details about:

   - sampling rate
   - filtering parameters
   - available channel types
   - bad channels
   - etc.


In [ ]:
print(raw.info)

`raw.info` is just a dictionary and like Python dictionaries has a `.keys()` method that shows all the available field names

In [ ]:
isinstance(raw.info, dict)
print(raw.info.keys())

So we can access its elements this way:

In [ ]:
raw.info['sfreq']  # Sampling frequency

In [ ]:
raw.info['bads']  # list of marked bad channels

Looking at the info dict we observe that data were already filtered.

Now, let's see what channels are present in the data. We simply have to take a loot at the `raw.ch_names` attribute.

In [ ]:
raw.ch_names

You can index it as a list

In [ ]:
raw.ch_names[42]

In [ ]:
raw.ch_names[:10]

Channel type of a specific channel

In [ ]:
mne.channel_type?

In [ ]:
channel_type = mne.channel_type(info=raw.info, idx=75)
print('Channel #75 is of type:', channel_type)

channel_type = mne.channel_type(info=raw.info, idx=320)
print('Channel #320 is of type:', channel_type)

`raw.info['chs']` contains all the details about the sensors (type, locations, coordinate frame etc.)

In [ ]:
len(raw.info['chs'])

In [ ]:
type(raw.info['chs'])

In [ ]:
raw.info['chs'][0]

In [ ]:
raw.info['chs'][330]

It is possible to rename channels using the [`rename_channels()`](https://mne.tools/stable/generated/mne.io.Raw.html#mne.io.Raw.rename_channels) method

In [ ]:
raw.rename_channels({"EOG 061": "blink detector"})

To visualize the sensor locations we can use [`plot_sensors()`](https://mne.tools/stable/generated/mne.io.Raw.html#mne.io.Raw.plot_sensors) method.

In [ ]:
raw.plot_sensors(kind='topomap', ch_type='mag');  # default topomap
raw.plot_sensors(ch_type='grad');
raw.plot_sensors(kind="3d", ch_type="all");

The [Working with sensor locations](https://mne.tools/stable/auto_tutorials/intro/40_sensor_locations.html#sphx-glr-auto-tutorials-intro-40-sensor-locations-py) tutorial describes how to read and plot sensor locations, and how MNE-Python handles physical locations of sensors.

# Extracting data from `Raw` objects

To access the data, just use the `[]` syntax as to access any element of a list, dict etc. MNE-Python also returns an array of times (in seconds) corresponding to the requested samples.

In [ ]:
start, stop = 0, 10
data, times = raw[:, start:stop]  # fetch all channels and the first 10 time points
print(data.shape)
print(times.shape)

In [ ]:
times

To extract data in a given time window

In [ ]:
sampling_freq = raw.info["sfreq"]
start_stop_seconds = np.array([11, 13])
start_sample, stop_sample = (start_stop_seconds * sampling_freq).astype(int)
channel_index = 0
raw_selection = raw[channel_index, start_sample:stop_sample]
print(raw_selection)

To get all data you can use the `get_data()` method

In [ ]:
data = raw.get_data()
print(data.shape)

# Visualizing raw data

Note : we will use the QT backend from matplotlib that will open a separate window.

In [ ]:
%matplotlib qt
fig = raw.plot()

In [ ]:
fig = raw.copy().pick_types(meg=False, eeg=True).plot()

# Filtering

In [ ]:
raw = mne.io.read_raw_fif(raw_fname, preload=True)

In [ ]:
raw_beta = raw.copy().filter(l_freq=13, h_freq=30, verbose=True)

In [ ]:
print(raw_beta.info)  # note the update of raw.info['lowpass'] and raw.info['highpass']

In [ ]:
raw_beta.plot()

In [ ]:
raw_beta.filter?

## Exercise
Plot the 10 first seconds of the stimutation channel `STI 014` just using matplotlib.

Tips:

- Find the channel index using `raw.ch_names.index('STI 014')`
- Get the data for this channel
- Plot it using `plt.plot`


# Define and read epochs

Let us now see how events are represented and used in MNE

### First, extract events.
The [`sample`](https://mne.tools/stable/documentation/datasets.html#sample-dataset) dataset includes experimental events recorded on stim channel `STI 014`. The events are parsed from this channel using [`mne.find_events()`](https://mne.tools/stable/generated/mne.find_events.html#mne.find_events) method:

In [ ]:
events = mne.find_events(raw, stim_channel='STI 014', verbose=True)
print(events.shape)
print(type(events))

LA - 1 - Response to left-ear auditory stimulus \
RA - 2 - Response to right-ear auditory stimulus \
LV - 3 - Response to left visual field stimulus \
RV - 4 - Response to right visual field stimulus \
smiley - 5 - Response to the smiley face \
button - 32 - Response triggered by the button press

In [ ]:
print(events[:5])  # events is a 2d array, (time, previous, trigger)

In [ ]:
len(events[events[:, 2] == 4])

In [ ]:
len(events)

## Plot events

In [ ]:
%matplotlib inline
fig = mne.viz.plot_events(events, sfreq=raw.info['sfreq'])

We can create an event Python dictionary to keep track of which Event ID corresponds to which experimental condition. The dictionary will be used to extract epochs from continuous data. The dictionary keys can contain `/` for grouping of sub-conditions. For example, if we want to pool all auditory trials, instead of merging Event IDs 1 and 2 using the `merge_events()` function, we can request for 'auditory' to select all epochs with Event IDs 1 and 2; requesting 'left' trials will select all epochs with Event IDs 1 and 3. 


In [ ]:
event_id = {"visual/left": 3, "visual/right": 4,
            "auditory/left": 1, "auditory/right": 2}

fig = mne.viz.plot_events(events, sfreq=raw.info['sfreq'], event_id=event_id)

The events can be visualized together with the raw data:

In [ ]:
raw.plot(
    events=events,
    start=5,
    duration=10,
    color="gray",
    event_color={1: "r", 2: "g", 3: "b", 4: "m", 5: "y", 32: "k"},
);

### Create epochs

In MNE-Python [`Epochs`](https://mne.tools/stable/generated/mne.Epochs.html#mne.Epochs) objects are a data structure for representing and analyzing equal-duration trials of the M/EEG signal. Epochs are most often used to represent data that is time-locked to repeated experimental events (such as stimulus onsets or subject button presses), but can also be used for storing sequential or overlapping frames of a continuous signal (e.g., for analysis of resting-state activity). \
Inside an Epochs object, the data are stored in an array of shape `(n_epochs, n_channels, n_times)`. 

First, define epochs parameters: start, stop, and baseline period of the epochs.

In [ ]:
tmin = -0.2  # start of each epoch (200ms before the trigger)
tmax = 0.5   # end of each epoch (500ms after the trigger)
baseline = (None, 0)  # from the first time instant to the trigger pulse

Define peak-to-peak (amplitude range) rejection parameters for gradiometers, magnetometers, and EOG:

In [ ]:
reject = dict(grad=4000e-13, mag=4e-12, eog=150e-6)  # this can be highly data dependent

In [ ]:
# we select MEG and EOG channels
picks_meg = mne.pick_types(raw.info, meg=True, eeg=False, eog=True,
                           stim=False, exclude='bads')

Extract epochs:

In [ ]:
epochs = mne.Epochs(raw, events, event_id, tmin, tmax, proj=True,
                    picks=picks_meg, baseline=baseline,
                    reject=reject)

In [ ]:
print(epochs)

Remove bad epochs based on the `reject` parameter we passed to `Epochs`.

In [ ]:
epochs.drop_bad()

See how epochs were dropped

In [ ]:
fig = epochs.plot_drop_log()

To access the data of some epochs use the [`get_data()`](https://mne.tools/stable/generated/mne.Epochs.html#mne.Epochs.get_data) method. \
`epochs_data` is a 3D array of dimension n_epochs x n_channels x n_time_points

In [ ]:
epochs_data = epochs.get_data()
type(epochs_data), epochs_data.shape

### Visualization Epochs

See [this page](https://mne.tools/stable/auto_tutorials/epochs/20_visualize_epochs.html) for options on how to visualize epochs. \
The `Epochs` object can be visualized (and browsed interactively) using its [`plot()`](https://mne.tools/stable/generated/mne.Epochs.html#mne.Epochs.plot) method



In [ ]:
%matplotlib qt

In [ ]:
epochs.plot(n_epochs=10, events=True)

In the plot above we can see heartbeat artifacts in the magnetometer channels, so before we continue let’s load ECG projectors from disk and apply them to the data:

In [ ]:
ecg_proj_file = data_path / "MEG" / "sample" / "sample_audvis_ecg-proj.fif"
ecg_projs = mne.read_proj(ecg_proj_file)
epochs.add_proj(ecg_projs)
epochs.apply_proj()

A convenient way to visualize many epochs simultaneously is to plot them as an image map, with each row of pixels in the image representing a single epoch, the horizontal axis representing time, and each pixel’s color representing the signal value at that time sample for that epoch.

In [ ]:
figs = epochs['auditory'].plot_image(combine='mean')

It is also possible to plot spectral power estimates across sensors as a scalp topography

In [ ]:
spectrum = epochs["visual/right"].compute_psd()
bands = {"10 Hz": 10, "15 Hz": 15, "20 Hz": 20, "10-20 Hz": (10, 20)}
spectrum.plot_topomap(bands=bands, vlim="joint", ch_type="grad")

## Average the epochs to get the evoked response (ERF/ERP)

[`Evoked`](https://mne.tools/stable/generated/mne.Evoked.html#mne.Evoked) objects typically store M/EEG signals that have been averaged over multiple epochs, which is a common technique for estimating stimulus-evoked activity. The data in an `Evoked` object are stored in an array of shape `(n_channels, n_times)`. \
We already have the `Epochs` object, so we can simply use its `average` method

In [ ]:
evoked = epochs.average()
print(evoked)

The information about the baseline period of `Epochs` is transferred to derived `Evoked` objects to maintain provenance as you process your data:

In [ ]:
print(f"Epochs baseline: {epochs.baseline}")
print(f"Evoked baseline: {evoked.baseline}")

In [ ]:
%matplotlib inline
fig = evoked.plot(spatial_colors=True)

This created an average across **all** conditions. Let's now estimate evoked responses for **individual** conditions.

In [ ]:
print(event_id)

In [ ]:
fig = epochs['auditory/left'].average().plot(spatial_colors=True)

The `plot()` methods for [`Raw`](https://mne.tools/stable/generated/mne.io.Raw.html#mne.io.Raw.plot), [`Epochs`](https://mne.tools/stable/generated/mne.Epochs.html#mne.Epochs.plot) and [`Evoked`](https://mne.tools/stable/generated/mne.Evoked.html#mne.Evoked.plot) objects, has many parameters for customizing the plot output, such as color-coding channel traces by scalp location, or plotting the global field power alongside the channel traces. 

## Accessing and indexing epochs by condition

Epochs can be indexed by integers or slices to select a subset of epochs but also with strings to select by conditions `epochs[condition]`

Remember `/` serves as a grouping operator. To calculate the evoked response across **all** "left" stimulations, do the following:

In [ ]:
fig = epochs['left'].average().plot(spatial_colors=True);  # note the legend

In [ ]:
# remember ...
event_id

In [ ]:
epochs[0]  # first epoch

In [ ]:
epochs[:10]  # first 10 epochs

In [ ]:
epochs['visual/left']  # epochs for the left visual condition

In [ ]:
epochs['visual']  # epochs for the visual condition (either left or right)

In [ ]:
epochs['left']

In event_id, `/` selects conditions in a hierarchical way, e.g. here, "auditory" vs. "visual", "left" vs. "right", and MNE can select them individually.

In [ ]:
%matplotlib qt

In [ ]:
evoked_auditory_left = epochs['auditory/left'].average().pick_types(meg='grad')
evoked_auditory_left.crop(None, 0.2) # Beginning of evoked until 0.2s after stimulus onset.
fig = evoked_auditory_left.plot(spatial_colors=True)
fig = evoked_auditory_left.plot(spatial_colors=True, gfp=True)

In the interactive session, the butterfly plots seen above can be click-dragged to select a time region, which will pop up a map of the average field distribution over the scalp for the selected time span.

## Visualize Topographies

The scalp topographies at specific times or time spans can be also generated by using the [`plot_topomap()`](https://mne.tools/stable/generated/mne.Evoked.html#mne.Evoked.plot_topomap) method


In [ ]:
fig = evoked.plot_topomap(ch_type='mag', times=[0.05, 0.1, 0.15])
fig = evoked.plot_topomap(ch_type='grad', times=[0.05, 0.1, 0.15])

In [ ]:
import numpy as np

times = np.linspace(0.05, 0.15, 5)
for ch_type in ('mag', 'grad'):
    fig = evoked.plot_topomap(times=times, ch_type=ch_type)

It is also possible to pass different time durations to average over for each time point. Passing a value of None will disable averaging for that time point:

In [ ]:
averaging_durations = [0.01, 0.02, 0.03, None, None]
fig = evoked.plot_topomap(
    ch_type="mag", times=times, average=averaging_durations
)

Joint plots combine butterfly plots with scalp topographies, and provide an excellent first-look at evoked data; by default, topographies will be automatically placed based on peak finding.


In [ ]:
figs = evoked.plot_joint()

But of course, you can also specify custom time points for the topomaps.

In [ ]:
figs = evoked.plot_joint(times=[0.1, 0.3])

Let's visualize topomaps for all experimental conditions.

In [ ]:
for condition in event_id:
    fig = epochs[condition].average().plot_topomap(times=[0.1, 0.15])

### Compute a contrast:

The function [`combine_evoked()`](https://mne.tools/stable/generated/mne.combine_evoked.html#mne.combine_evoked) computes a weighted sum of the `Evoked` objects given to it

In [ ]:
evoked1 = epochs['auditory/left'].average()
evoked2 = epochs['auditory/right'].average()

contrast = mne.combine_evoked([evoked1, evoked2], weights=[1, -1])

Note that this combines evokeds taking into account the number of averaged epochs (to scale the noise variance)

In [ ]:
print(evoked1.nave)  # average of 55 epochs
print(evoked2.nave)  # average of 61 epochs
print(contrast.nave)  # average of 116 epochs

In [ ]:
print(contrast)

In [ ]:
fig = evoked1.plot_joint()

### EXERCISE
- Extract Epochs restricted to magnetometers on unfiltered data (`sample_audvis_raw.fif`)
- Construct epochs with a whole-epoch baseline. Then, high-pass filter raw data with a 1 Hz cutoff, construct epochs from that. Compare the resulting Evokeds (filter vs. baseline)
- Plot the difference between all *visual* and all *auditory* stimulus presentations
- Recompute everything for EEG

<div class="alert alert-block alert-info">
<b>Note:</b> For more details look at the following tutorials: <br>
    <a href="https://mne.tools/stable/auto_tutorials/raw/index.html" target="_blank">Working with continuous data </a> <br>
    <a href="https://mne.tools/stable/auto_tutorials/epochs/index.html" target="_blank">Segmenting continuous data into epochs</a> <br>
    <a href="https://mne.tools/stable/auto_tutorials/evoked/index.html" target="_blank">Estimating evoked responses</a>
</div>
